In [ ]:
import pandas as pd
import numpy as np
import geopandas as gpd

import matplotlib.pyplot as plt
import contextily as ctx
from shapely.geometry import Point

pd.set_option('display.max_columns', 999)
import warnings
warnings.filterwarnings('ignore')

In [ ]:
path = gpd.datasets.get_path('naturalearth_lowres')
df = gpd.read_file(path)

In [ ]:
data = pd.read_csv('../input/argentina-properati-listings-dataset-20202021/ar_properties.csv')

In [ ]:
data.head()

In [ ]:
size = data.groupby('l1').size().to_frame('listings')
translation = {'Argentina':'Argentina', 'Uruguay':'Uruguay',
               'Brasil':'Brazil', 'Estados Unidos':'United States of America'}
size.index = size.index.map(translation)

countries = ['Argentina', 'Uruguay', 'Brazil', 'United States of America']

df = df.set_index('name').join(size).reset_index()

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize = (15,10), dpi = 600)


df[df['continent'].str.contains('America')].to_crs(epsg = 3857).boundary.plot(figsize = (15,10), ax = ax1, color = 'black', lw = 0.5)
df[df['name'].isin(countries)].to_crs(epsg = 3857).plot(ax = ax1, color = 'green')


ax1.axis('off')
ax1.set_title('Countries available')

df[df['name'].isin(countries)].set_index('name')['listings'].sort_values(
    ascending = False
).plot.bar(
    ax = ax2, color = 'green', rot = 0)

countries = df[df['name'].isin(countries)].set_index('name')['listings'].sort_values(
    ascending = False
).index.tolist()

values = df[df['name'].isin(countries)].set_index('name')['listings'].sort_values(
    ascending = False
).values.tolist()

ticks = range(len(countries))

for i, val in enumerate(values):

    label = "{:,.0f}".format(val)
    ax2.annotate(label, (ticks[i], values[i]), 
                 ha = 'center', 
                 xytext=(0,10), 
                 textcoords='offset points')

ax2.set_xlabel('Country')
ax2.set_title('Number of listings by country')